# Transformer

自从 2017 年 Google 发布《Attention is All You Need》之后，各种基于 Transformer 的模型和方法层出不穷。尤其是 2018 年，OpenAI 发布的 GPT 和 Google 发布的 BERT 模型在几乎所有 NLP 任务上都取得了远超先前最强基准的性能。

Transformer 模型之所以如此强大，是因为它抛弃了之前广泛采用的循环网络和卷积网络，而采用了一种特殊的结构——注意力机制 (Attention) 来建模文本。

## Attention
> 来源：https://transformers.run/back/attention/

NLP 神经网络模型的本质就是对输入文本进行编码，**常规的做法是首先对句子进行分词**，然后将每个词语 (token) 都转化为对应的词向量 (token embeddings)，这样文本就转换为一个由词语向量组成的矩阵 $X = (x_1, x_2, \cdots, x_n)$。其中 $x_i$ 就表示第 i 个词语的词向量，维度为 d。

在 Transformer 模型提出之前，对 token 序列 $X$ 的常规编码方式是通过循环网络 (RNNs) 和卷积网络 (CNNs)。

> RNN
> RNN（例如 LSTM）的方案很简单，每一个词语 $x_t$ 对应的编码结果 $y_t$ 通过递归地计算得到：
> $$
> y_t = f(y_{t-1}, x_t)
> $$
>
> RNN 的序列建模方式虽然与人类阅读类似，但是递归的结构导致其无法并行计算，因此速度较慢。而且 RNN 本质是
> 一个马尔科夫决策过程，难以学习到全局的结构信息；一般使用双向 RNN


> CNN
> CNN 则通过滑动窗口基于局部上下文来编码文本，例如核尺寸为 3 的卷积操作就是使用每一个词自身以及前一个和后
> 一个词来生成嵌入式表示：
> $$
> y_t = f(x_{t-1}, x_t, x_{t+1})
> $$
> 
> CNN 能够并行地计算，因此速度很快，但是由于是通过窗口来进行编码，所以更侧重于捕获局部信息，难以建模长距离> 的语义依赖。

Google《Attention is All You Need》提供了第三个方案：直接使用 Attention 机制编码整个文本。相比 RNN 要逐步递归才能获得全局信息（因此一般使用双向 RNN），而 CNN 实际只能获取局部信息，需要通过层叠来增大感受野，Attention 机制一步到位获取了全局信息。

### Scaled Dot-Product Attention
虽然 Attention 有许多种实现方式，但是最常见的还是 Scaled Dot-product Attention。
Scaled Dot-product Attention 共包含 2 个主要步骤：

1. 计算注意力权重
   使用某种相似度函数度量每一个 query 向量和所有 key 向量之间的关联程度。
   特别的，Scale Dot-Product 使用点积作为相似度函数，这样相似的 queries 和 keys 会具有较大的点积。由于点积可以产生任意大的数字，这会破坏训练过程的稳定性。因此注意力分数还需要乘以一个缩放因子来标准化它们的方差，然后用一个 softmax 标准化。
2. 更新 token embeddings
   将权重与对应的 value 向量 相乘以获得第 i 个 query 向量更新后的语义表示

形式化的表示为：

![](assets/1.png)


下文使用 pytorch 手动实现 Scaled Dot-Product，但是再实现之前，先要将词语转换为对应的 embedding

#### Tokenize Module

In [17]:
# 首先需要将文本分词为词语 (token) 序列，然后将每一个词语转换为对应的词向量 (token embedding)。Pytorch 提供了 torch.nn.Embedding 层来完成该操作，即构建一个从 token ID 到 token embedding 的映射表

import torch.nn as nn
from transformers import AutoConfig, AutoTokenizer

class Tokenizer():
    def __init__(self, model_name, text, add_special_tokens=True, max_length=512, return_tensors='pt'):
        self.model_name = model_name
        self.text = text
        self.tokenizer = AutoTokenizer.from_pretrained(model_name) # 加载预训练标记器
        self.config = AutoConfig.from_pretrained(model_name) # 加载预训练配置

        self.inputs = self.tokenizer(self.text, add_special_tokens=add_special_tokens, max_length=max_length, return_tensors=return_tensors) # 使用加载的标记器对输入文本进行标记，并生成模型输入
        self.token_embedding = nn.Embedding(self.config.vocab_size, self.config.hidden_size) # 嵌入层，用于将标记映射为密集嵌入向量
    def ids_to_embedding(self):
        return self.token_embedding(self.inputs.input_ids)

#### Tokenize Example

In [18]:
model_name = 'bert-base-uncased'
text = 'time flies like an arrow'

tokenizer = Tokenizer(model_name, text, add_special_tokens=True) # add_special_tokens=True 表示在标记化文本中添加特殊标记 [CLS] 和 [SEP]，分别代表句子的开始（classification）和分隔（seperator） 
print(tokenizer.inputs) # 文本标记，一个字典，其中 input_ids 是标记化文本的 ID，attention_mask 是注意力掩码，token_type_ids 是标记类型 ID
print(tokenizer.token_embedding)
print(tokenizer.ids_to_embedding()) # 将标记 ID 转换为嵌入向量
print(tokenizer.ids_to_embedding().shape)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


{'input_ids': tensor([[  101,  2051, 10029,  2066,  2019,  8612,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1]])}
Embedding(30522, 768)
tensor([[[ 1.2687,  0.5931, -0.6031,  ...,  0.9283,  0.7474,  1.0823],
         [-0.4959,  0.4953,  0.4934,  ..., -1.4176, -1.2415,  2.0562],
         [-0.8503,  0.5852,  0.0258,  ...,  1.5025,  0.8617,  0.1098],
         ...,
         [ 0.5211,  0.2322,  1.0804,  ..., -0.6803, -1.5351,  0.2344],
         [-0.4931, -1.1086,  0.3540,  ..., -0.0685, -2.2003,  0.6082],
         [ 1.2116, -0.4722,  0.1345,  ..., -0.0192, -0.3404, -0.1030]]],
       grad_fn=<EmbeddingBackward0>)
torch.Size([1, 7, 768])


#### A Simple Scaled Dot-Product Attention

将上述的操作封装为函数

In [19]:
from torch import bmm
import torch.nn.functional as F
from math import sqrt

# batch matrix-multiplication 函数（Batch Matrix Multiplication）。它用于执行两个批次（batch）矩阵的乘法操作。
def scaled_dot_product_attention(query, key, value, query_mask=None, key_mask=None, mask=None):
    dim_k = query.size(-1) # 获取 query 的最后一个维度，即嵌入维度
    scores = bmm(query, key.transpose(-1, -2)) / sqrt(dim_k) # 计算 query 和 key 的点积，并缩放
    # print(scores.shape)
    if query_mask is not None and key_mask is not None:
        # print(query_mask.shape)
        # print(key_mask.shape)
        # print(query_mask.unsqueeze(-1).shape)
        mask = bmm(query_mask.unsqueeze(-1), key_mask.unsqueeze(1)) # 生成通用掩码
        # print(mask.shape)
    if mask is not None:
        scores = scores.masked_fill(mask == 0, float('-inf')) # 通用掩码不为空的话，将 scores 中对应位置的值替换为负无穷，因为填充 (padding) 字符不应该参与计算，因此将对应的注意力分数设置为负无穷，保证其 softmax 后的值为 0

    weights = F.softmax(scores, dim=-1)
    return bmm(weights, value)

In [20]:
Q = K = V = tokenizer.ids_to_embedding()
sdpa = scaled_dot_product_attention(Q, K, V, query_mask=tokenizer.inputs['attention_mask'], key_mask=tokenizer.inputs['attention_mask'])

#### ps:关于 unsqueeze 函数和掩码

In [21]:
import torch
# 生成一维掩码张量
mask = torch.tensor([1, 0, 1])
print(mask)

# 在第一个维度插入新的维度，变为 (1, seq_length)
new_mask = mask.unsqueeze(0)
print(new_mask)

# 在第二个维度插入新的维度，变为 (seq_length, 1)
new_mask = mask.unsqueeze(1)
print(new_mask)


tensor([1, 0, 1])
tensor([[1, 0, 1]])
tensor([[1],
        [0],
        [1]])


## MultiHead Attention
注意！上面的做法会带来一个问题：当 Q 和 K 序列相同时，注意力机制会为上下文中的相同单词分配非常大的分数（点积为 1），而在实践中，相关词往往比相同词更重要。例如对于上面的例子，只有关注“time”和“arrow”才能够确认“flies”的含义。

因此，多头注意力 (Multi-head Attention) 出现了！
Multi-head Attention 实质上就是拼接多个注意力头的输出，多做几次 Scaled Dot-product Attention

![](assets/2.png)

### A Single Attention Head

将上文的 Scaled Dot-Product Attention 封装成类，就是一个简单的 Attention Head

In [22]:
import torch.nn as nn

class AttentionHead(nn.Module):
    def __init__(self, embed_dim, head_dim):
        super(AttentionHead, self).__init__()
        self.Q = nn.Linear(embed_dim, head_dim)
        self.K = nn.Linear(embed_dim, head_dim)
        self.V = nn.Linear(embed_dim, head_dim)

    def forward(self, query, key, value, query_mask=None, key_mask=None, mask=None):
        Q = self.Q(query)
        K = self.K(key)
        V = self.V(value)
        return scaled_dot_product_attention(Q, K, V, query_mask, key_mask, mask)

### MultiHead Attention

每个头都会初始化三个独立的线性层，负责将 Q, K, V 序列映射到尺寸为 [batch_size, seq_len, head_dim] 的张量，其中 head_dim 是映射到的向量维度。
> 实践中一般将 head_dim 设置为 embed_dim 的因数，这样 token 嵌入式表示的维度就可以保持不变，例如 BERT 有 12 个注意力头，因此每个头的维度被设置为 768 / 12 = 64

最后只需要拼接多个注意力头的输出就可以构建出 Multi-head Attention 层了（这里在拼接后还通过一个线性变换来生成最终的输出张量）

In [25]:
class MultiHeadAttention(nn.Module):
    def __init__(self, config):
        super(MultiHeadAttention, self).__init__()
        embed_dim = config.hidden_size
        num_heads = config.num_attention_heads
        head_dim = embed_dim // num_heads
        self.heads = nn.ModuleList([AttentionHead(embed_dim, head_dim) for _ in range(num_heads)])
        self.output_linear = nn.Linear(embed_dim, embed_dim)

    def forward(self, query, key, value, query_mask=None, key_mask=None, mask=None):
        head_outputs = [head(query, key, value, query_mask, key_mask, mask) for head in self.heads]
        outputs = torch.cat(head_outputs, dim=-1)
        outputs = self.output_linear(outputs)
        return outputs

### MultiHead Attention Test

这里使用 BERT-base-uncased 模型的参数初始化 Multi-head Attention 层，并且将之前构建的输入送入模型以验证是否工作正常：

In [27]:
model_name = 'bert-base-uncased'
text = 'time flies like an arrow'


tokenizer = Tokenizer(model_name, text, add_special_tokens=True)
multihead_attn = MultiHeadAttention(tokenizer.config)
Q = K = V = tokenizer.ids_to_embedding()
attn_output = multihead_attn(Q, K, V, query_mask=tokenizer.inputs['attention_mask'],key_mask=tokenizer.inputs['attention_mask'])
print(attn_output.size())

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


torch.Size([1, 7, 768])


## Transformer Encoder

![](assets/3.png)


Encoder 负责将输入的词语序列转换为词向量序列，Decoder 则基于 Encoder 的隐状态来迭代地生成词语序列作为输出，每次生成一个词语。

- 输入的词语首先被转换为词向量。由于注意力机制无法捕获词语之间的位置关系，因此还通过 positional embeddings 向输入中添加位置信息；
- Encoder 由一堆 encoder layers (blocks) 组成，类似于图像领域中的堆叠卷积层。同样地，在 Decoder 中也包含有堆叠的 decoder layers；
- Encoder 的输出被送入到 Decoder 层中以预测概率最大的下一个词，然后当前的词语序列又被送回到 Decoder 中以继续生成下一个词，重复直至出现序列结束符 EOS 或者超过最大输出长度。

### Feedforward Layer

Transformer Encoder/Decoder 中的前馈子层实际上就是两层全连接神经网络，它单独地处理序列中的每一个词向量，也被称为 position-wise feed-forward layer。常见做法是让第一层的维度是词向量大小的 4 倍，然后以 GELU 作为激活函数。

In [ ]:
class FeedForwardLayer(nn.Module):
    pass

In [ ]:
feed_forward = FeedForwardLayer(tokenizer.config)
ff_output = feed_forward(attn_output)
print(ff_output.size())

### Layer Normalization


### Positional Embeddings

## Transformer Decoder